In [2]:
import warnings


import numpy as np
import pandas as pd

from datasketch import MinHash, MinHashLSH


warnings.filterwarnings("ignore")

In [ ]:
events = pd.read_csv("./data/events.csv")

# Создадим датафреймы на основе предоставленных данных
properties_1 = pd.read_csv("./data/item_properties_part1.csv")
properties_2 = pd.read_csv("./data/item_properties_part2.csv")
item_properties = pd.concat([properties_1, properties_2])

# Приведем к временному формату данные из timestapm

item_properties["timestamp"] = pd.to_datetime(item_properties["timestamp"], unit="ms")

In [16]:
# Функция для формирования характеристик товара
def item_similarity(raw_df: pd.DataFrame, time_border: int = 7) -> pd.DataFrame:
    # Поработаем со свойствами товаров, сначала приведем property к int64,
    # как было указано в property лежат категории + хеши остальных свойств,
    # следовательно переведя в числовой формат и перехватив ошибку в нан мы должны
    # оставить только категории.

    item_properties = raw_df.copy()

    item_properties["timestamp"] = pd.to_datetime(
        item_properties["timestamp"], unit="ms"
    )

    item_properties = item_properties[
        (item_properties["property"] != "categoryid")
        & (item_properties["property"] != "available")
    ]

    item_properties = item_properties[
        item_properties["timestamp"].dt.month < time_border
    ]

    item_properties = item_properties.drop(columns=["timestamp", "property"])

    item_properties = item_properties.drop_duplicates()

    # Удалим пропуски
    item_properties.dropna(inplace=True)

    # Токенезируем value и переименуем во что-то более вразумительное
    item_properties = item_properties.rename(columns={"value": "charac"})

    # Создадим функцию для токенизации
    def item_prop_tokenizer(prop):
        prop = prop.replace("n", "")
        numeric_list = []
        for x in prop.split():
            try:
                numeric_list.append(float(x))
            except ValueError:
                numeric_list.append(float(0))
        return numeric_list

    item_properties["charac"] = item_properties["charac"].apply(item_prop_tokenizer)

    # Соберем все данные о характеристиках товара в один столбец
    items_feats = pd.DataFrame(
        item_properties.groupby(["itemid"])["charac"].agg(list)
    ).reset_index()

    # Создадим сет характеристик для каждого id товара
    def remove_empty_and_none(x):
        if x is None or x == "":
            return None
        else:
            flattened_array = np.concatenate(x)
            return set(flattened_array) if flattened_array.size > 0 else None

    items_feats["features_set"] = items_feats["charac"].apply(remove_empty_and_none)

    items_feats.drop(["charac"], axis=1, inplace=True)

    return items_feats

In [17]:
test = item_similarity(item_properties)

In [34]:
test[test["itemid"] == 356475]

,itemid,features_set
310858,356475,"{707081.0, 403210.0, 296717.0, 1065616.0, 1218..."


In [48]:
x = test[test["itemid"] == 78144]


In [19]:
# Пробуем через datasketch ибо annoy занимает дофига места в моем случае.
# Инициализируем LSH с порогом и количеством перестановок:
lsh = MinHashLSH(threshold=0.5, num_perm=128)

# 	Создаем и добавляем объекты MinHash:
for itemid, charac in zip(test["itemid"], test["features_set"]):
    minhash = MinHash(num_perm=128)
    for d in charac:
        minhash.update(str(d).encode("utf-8"))
    lsh.insert(str(itemid), minhash)

In [45]:
events

,timestamp,visitorid,event,itemid,transactionid
0,2015-06-02 05:02:12.117,257597,view,355908,NaN
1,2015-06-02 05:50:14.164,992329,view,248676,NaN
2,2015-06-02 05:13:19.827,111016,view,318965,NaN
3,2015-06-02 05:12:35.914,483717,view,253185,NaN
4,2015-06-02 05:02:17.106,951259,view,367447,NaN
...,...,...,...,...,...
2756096,2015-08-01 03:13:05.939,591435,view,261427,NaN
2756097,2015-08-01 03:30:13.142,762376,view,115946,NaN
2756098,2015-08-01 02:57:00.527,1251746,view,78144,NaN
2756099,2015-08-01 03:08:50.703,1184451,view,283392,NaN


In [47]:
events[(events["event"] == "transaction") & (events["itemid"] == 78144)]

,timestamp,visitorid,event,itemid,transactionid
131402,2015-06-07 14:40:37.376,659301,transaction,78144,16499.0
2374068,2015-07-16 20:37:35.464,1090856,transaction,78144,12245.0


In [51]:
# Попробуем найти похожие товары
query_item = x["features_set"].values
query_minhash = MinHash(num_perm=128)
for d in query_item:
    query_minhash.update(str(d).encode("utf-8"))

result = lsh.query(query_minhash)
top_3_results = result[:3]  # Выбираем первые 3 элементов
print("Похожие элементы:", top_3_results)
print(len(result))

Похожие элементы: []
0
